<a href="https://colab.research.google.com/github/deltorobarba/chemistry/blob/main/H2O_quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Water Molecule (H₂O) - Quantum Computing for Ground-state energy estimation**

In [ ]:
!pip install matplotlib openfermion openfermionpyscf pennylane cirq openfermioncirq -q

[IonQ: Ground-state energy estimation of the water molecule on a trapped ion quantum computer](https://ionq.com/links/Water_Molecule_VQE_Simulation.pdf)

https://pennylane.ai/qml/demos/tutorial_quantum_chemistry/

In [ ]:
# Quantum Computing for Ground-state energy estimation
# Variational Quantum Eigensolver

!pip install pennylane -q

import pennylane as qml
from pennylane import numpy as np

# Define the molecular system
symbols = ['O', 'H', 'H']
# The coordinates should be provided in atomic units (Bohr)
coordinates = np.array([0.0, 0.0, 0.0, 1.432, 1.107, 0.0, -1.432, 1.107, 0.0])  # Approximate atomic units

# Generate the molecular Hamiltonian using PennyLane's quantum chemistry module
H, qubits = qml.qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=0,
    mult=1,  # Singlet state
    basis='sto-3g',  # Simple minimal basis set
    active_electrons=4,  # 2 electrons from hydrogen and 2 from oxygen (for a minimal example)
    active_orbitals=4,  # Adjust as needed for accuracy
    mapping='jordan_wigner'  # Fermion-to-qubit mapping
)

# Define the quantum device
dev = qml.device('default.qubit', wires=qubits)

# Define the ansatz (parameterized quantum circuit)
def ansatz(params):
    for i in range(qubits):
        qml.RY(params[i], wires=i)
    for i in range(qubits - 1):
        qml.CNOT(wires=[i, i + 1])

# Define the cost function (expectation value of the Hamiltonian)
@qml.qnode(dev)
def cost_fn(params):
    ansatz(params)
    return qml.expval(H)

# Perform the VQE optimization
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)
params = np.random.randn(qubits)
for i in range(100):
    params = optimizer.step(cost_fn, params)

# Extract the ground state energy
ground_state_energy = cost_fn(params)

print("Ground state energy of H2O molecule:", ground_state_energy)

Ground state energy of H2O molecule: -73.8199543781253


In your quantum algorithm to compute the ground state energy of the water molecule using the Variational Quantum Eigensolver (VQE), the number of qubits required depends on the number of active orbitals and the fermion-to-qubit mapping used.

**How the Number of Qubits is Determined:**
- **Active orbitals:** In your example, you have specified `active_orbitals=4`. This means you're choosing 4 molecular orbitals to describe the electronic structure of the water molecule.
- **Qubit mapping (Jordan-Wigner transformation):** This transformation maps each spin-orbital to one qubit. For each active orbital, there are two spin states (spin-up and spin-down), so you need \( 2 \times \text{{active orbitals}} \) qubits.

Thus, for your setup:
$
\text{{Number of qubits}} = 2 \times \text{{active orbitals}} = 2 \times 4 = 8 \, \text{{qubits}}.
$

Therefore, you are using **8 qubits** in this quantum algorithm for the water molecule. This is determined by the 4 active orbitals, where each orbital has two spin states (requiring two qubits per orbital) under the Jordan-Wigner mapping.

**Example with Cirq and OpenFermion**

1. Define the water molecule's geometry
2. Generate the molecular Hamiltonian
3. Use Cirq for the quantum circuit simulation
4. Run the VQE optimization to find the ground state energy


Notes:
* **Molecular Geometry**: The water molecule's geometry is defined in atomic units (Bohr), just as in the PennyLane example.
* **Basis Set**: A minimal basis set `'sto-3g'` is used to reduce the number of qubits.
* **Jordan-Wigner Transformation**: The Fermionic Hamiltonian is mapped to a qubit Hamiltonian using the Jordan-Wigner transformation (which is also available in PennyLane).
* **Ansatz**: The `SwapNetworkTrotterAnsatz` is used in this example. You can experiment with different ansätze such as `Unitary Coupled Cluster (UCC)` or others.
* **Optimizer**: Cirq’s `SimulatedAnnealing` optimizer is used for VQE optimization. You can change this to other optimizers based on your preference (e.g., gradient-based optimizers).


- This code uses OpenFermion and Cirq, both of which integrate seamlessly for quantum chemistry and VQE tasks.
- For accuracy, you might want to run this on a quantum device or an advanced simulator like Google’s `Quantum Engine` when scaling up to larger molecules.
- The number of qubits will depend on the basis set and active space chosen (here, `sto-3g` is minimal).

In [ ]:
import cirq
import numpy as np
from openfermion import MolecularData
from openfermion.transforms import jordan_wigner, get_sparse_operator
from openfermionpyscf import run_pyscf
from scipy.sparse.linalg import eigsh  # For finding the ground state energy

# Step 1: Define the molecular system (H2O)
geometry = [('O', (0.0, 0.0, 0.0)), ('H', (1.432, 1.107, 0.0)), ('H', (-1.432, 1.107, 0.0))]
basis = 'sto-3g'
multiplicity = 1
charge = 0

# Step 2: Create the MolecularData object
molecule = MolecularData(geometry, basis, multiplicity, charge)

# Step 3: Run the quantum chemistry calculation using PySCF
molecule = run_pyscf(molecule)

# Step 4: Get the qubit Hamiltonian using Jordan-Wigner transformation
qubit_hamiltonian = jordan_wigner(molecule.get_molecular_hamiltonian())

# Convert the QubitOperator to a sparse matrix
sparse_hamiltonian = get_sparse_operator(qubit_hamiltonian)

# Step 5: Get the ground state energy using scipy's sparse eigenvalue solver
# The lowest eigenvalue corresponds to the ground state energy
ground_state_energy, ground_state = eigsh(sparse_hamiltonian, k=1, which='SA')
ground_state_energy = ground_state_energy[0]
print(f"Classical ground state energy: {ground_state_energy}")

# Step 6: Define the quantum circuit (ansatz) using Cirq
qubits = cirq.LineQubit.range(sparse_hamiltonian.shape[0])

# Define a simple ansatz for VQE
def create_ansatz(params):
    circuit = cirq.Circuit()
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.rx(params[i])(qubit))
    for i in range(len(qubits) - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))
    return circuit

# Define the cost function
def vqe_cost(params):
    ansatz = create_ansatz(params)
    simulator = cirq.Simulator()
    result = simulator.simulate(ansatz)
    energy = np.real(np.dot(result.final_state_vector.conj().T, sparse_hamiltonian @ result.final_state_vector))
    return energy

# Step 7: Perform the VQE optimization using SciPy optimizer
from scipy.optimize import minimize

# Initial parameters for the ansatz
initial_params = np.random.randn(len(qubits))

# Minimize the VQE cost function
result = minimize(vqe_cost, initial_params, method='BFGS')
vqe_energy = result.fun

print(f"VQE ground state energy: {vqe_energy}")


ImportError: cannot import name 'get_sparse_operator' from 'openfermion.transforms' (/usr/local/lib/python3.10/dist-packages/openfermion/transforms/__init__.py)